In [ ]:
import json
import numpy as np
import osmnx as ox
import pandas as pd
from ast import literal_eval
from copy import deepcopy
from os.path import join
from pathlib import Path
from setup_paths import paths
from shutil import copyfile

In [ ]:
rng = np.random.default_rng()

In [ ]:
save = "cph_tud"

In [ ]:
edge_dtypes = {
    'NormalInfraType': int,
    'UpgradedInfraType': int,
    'IntersectionDelay': int,
    'seg_id': int,
    'cost': float,
}
g = ox.load_graphml(join(paths["input_folder"], save, f"{save}.graphml"), edge_dtypes=edge_dtypes)
for _, _, data in g.edges(data=True):
    data["Active_Basis"] = literal_eval(data["Active_Basis"])
    data["ex_inf"] = literal_eval(data["ex_inf"])
    data["blocked"] = literal_eval(data["blocked"])
    data["bike_highway"] = literal_eval(data["bike_highway"])
cost = pd.read_csv(join(paths["data_dir"], "raw", "CPH", "SegmentCosts.csv"))

In [ ]:
with open(join(paths["input_folder"], save, f"{save}_demand.json"), "r") as f:
    demand = json.load(f)

In [ ]:
node_data = pd.read_csv(join(paths["data_dir"], "raw", "CPH", "NodesData.csv"))
zone_data = pd.read_csv(join(paths["data_dir"], "raw", "CPH", "CentroidsData.csv"), sep=",", dtype={"NodeId": int, "ZoneId": int})
node_data = pd.merge(node_data, zone_data, how="left", on="NodeId")
node_data["ZoneId"] = node_data["ZoneId"].fillna(-1)
node_data = node_data.astype({"ZoneId": "int32"})
node_data["NodeId"] += 1

In [ ]:
demand_df =  pd.read_csv(join(paths["data_dir"], "raw", "CPH", "ODData_Complete.csv"), sep=",",
                        dtype={"FromZoneId": int, "ToZoneId": int, "TravelerType": int, "value": float,
                               "gc_car": float, "gc_pt": float, "gc_walk": float,
                               "gamma_car": float, "gamma_car_passenger": float, "gamma_pt": float, "gamma_walk": float,
                               "p_other": float}
                        )

In [ ]:
def demand_to_od_df(demand_dict, old_demand_csv, nodes):
    new_demand_df = deepcopy(old_demand_csv)
    
    new_demand_df["FromNodeId"] = new_demand_df.FromZoneId.map(nodes.set_index("ZoneId")["NodeId"].to_dict())
    new_demand_df["ToNodeId"] = new_demand_df.ToZoneId.map(nodes.set_index("ZoneId")["NodeId"].to_dict())
    new_demand_df["FromTo"] = list(zip(new_demand_df.FromNodeId, new_demand_df.ToNodeId))
    new_demand_df["value"] = [demand_dict[row["FromTo"]][str(row["TravelerType"])]["number_of_users"] for index, row in new_demand_df.iterrows()]
    
    new_demand_df.drop(["FromNodeId", "ToNodeId", "FromTo"], axis=1, inplace=True)
    
    return new_demand_df

In [ ]:
def noisy_demand(demand_dict, delta): 
    noisy_demand_dict = {literal_eval(od): deepcopy(trips) for od, trips in demand_dict.items()}
    
    nodes = set()
    for od in noisy_demand_dict.keys():
        nodes.add(od[0])
        nodes.add(od[1])
    nodes_noisy = {node: rng.uniform(1 - delta, 1 + delta) for node in nodes}
    
    # traveler type: cyclist types with speed types increasing: 1-3 normal, 4-6 e-bike, 7-9 s-pedelec
    for od, noisy_trips in noisy_demand_dict.items():
        for c_type in noisy_trips.keys():
            demand_old = demand_dict[f"{od}"][c_type]["number_of_users"]
            noise_factor = (nodes_noisy[od[0]] + nodes_noisy[od[1]]) / 2
            noisy_demand_dict[od][c_type]["number_of_users"] = demand_old * noise_factor
    return noisy_demand_dict, nodes_noisy

In [ ]:
def noisy_speeds(speeds_array, delta):
    high = 1 + delta
    low = 1 - delta
    
    speeds = np.array([
        [[13.6, 15.1, 16.6], [16.3, 17.8, 19.3], [19.1, 20.8, 22.5]],
        [[15.6, 17.1, 18.6], [18.3, 19.8, 21.3], [21.1, 22.8, 24.5]],
        [[22.6, 24.1, 25.6], [25.3, 26.8, 28.3], [27.3, 29.8, 31.5]],
    ])
    noisy_speeds_array = np.zeros((3, 3, 3))
    noisy_speeds_dict = {"super_speeds": np.zeros(9), "bp_speeds": np.zeros(9), "street_speeds": np.zeros(9)}
    
    bike_types = ["bike", "e-bike", "s-pedelec"]
    speed_types = ["slow", "medium", "fast"]
    
    for i in range(9):
        bike_type = i // 3
        speed_type = i % 3
        cyclist_type = f"{bike_types[bike_type]} {speed_types[speed_type]}"
        
        new_street_speed = speeds[bike_type][speed_type][0] * rng.uniform(low, high)
        new_bp_speed = speeds[bike_type][speed_type][1] * rng.uniform(low, high)
        new_super_speed = speeds[bike_type][speed_type][2] * rng.uniform(low, high)
        
        while new_super_speed < new_bp_speed or new_bp_speed < new_street_speed or new_super_speed < new_street_speed \
                or new_super_speed < noisy_speeds_array[max(bike_type-1,0)][speed_type][2] or new_bp_speed < noisy_speeds_array[max(bike_type-1,0)][speed_type][1] or new_street_speed < noisy_speeds_array[max(bike_type-1,0)][speed_type][0] \
                or new_super_speed < noisy_speeds_array[bike_type][max(speed_type-1,0)][2] or new_bp_speed < noisy_speeds_array[bike_type][max(speed_type-1,0)][1] or new_street_speed < noisy_speeds_array[bike_type][max(speed_type-1,0)][0]:
            new_super_speed = speeds[bike_type][speed_type][0] * rng.uniform(low, high)
            new_bp_speed = speeds[bike_type][speed_type][1] * rng.uniform(low, high)
            new_street_speed = speeds[bike_type][speed_type][2] * rng.uniform(low, high)
        
        noisy_speeds_array[bike_type][speed_type][0] = new_street_speed
        print(f"{cyclist_type} on streets: {new_street_speed:2.3f}")
        noisy_speeds_array[bike_type][speed_type][1] = new_bp_speed
        print(f"{cyclist_type} on bike paths: {new_bp_speed:2.3f}")
        noisy_speeds_array[bike_type][speed_type][2] = new_super_speed
        print(f"{cyclist_type} on highways: {new_super_speed:2.3f}")
    
        noisy_speeds_dict["street_speeds"][i] = new_street_speed
        noisy_speeds_dict["bp_speeds"][i] = new_bp_speed
        noisy_speeds_dict["super_speeds"][i] = new_super_speed
    
    noisy_speeds_dict = {k: v.tolist() for k, v in noisy_speeds_dict.items()}
    return noisy_speeds_dict

In [ ]:
def noisy_costs(g, cost_df, delta):  
    noisy_cost_df = deepcopy(cost_df)
    noisy_g = deepcopy(g)
    
    noisy_cost_df["ConstructionCosts"] = noisy_cost_df["ConstructionCosts"] * rng.uniform(1 - delta, 1 + delta, len(noisy_cost_df["ConstructionCosts"]))
    noisy_cost_df["MaintenanceCosts"] = noisy_cost_df["MaintenanceCosts"] * rng.uniform(1 - delta, 1 + delta, len(noisy_cost_df["MaintenanceCosts"]))
    
    for _, _, data in noisy_g.edges(data=True):
        if data["seg_id"] != -1:
            data["cost"] = noisy_cost_df.loc[noisy_cost_df["SegmentId"]==data["seg_id"], "ConstructionCosts"].values[0]
            
    return noisy_g, noisy_cost_df

In [ ]:
def noisy_utilities(vod, vot, delta):  
    vod_noisy = {k: v * rng.uniform(1 - delta, 1 + delta) for k, v in vod.items()}
    vot_noisy = vot * rng.uniform(1 - delta, 1 + delta)
    
    return vod_noisy, vot_noisy

In [ ]:
def gen_noisy_costs(d, clean_costs, noisy_cost_save):
    noisy_cost_folder = join(paths["input_folder"], noisy_cost_save)
    Path(noisy_cost_folder).mkdir(parents=True, exist_ok=True)
    
    g_noisy, cost_noisy = noisy_costs(g, clean_costs, d)
    ox.save_graphml(g_noisy, join(noisy_cost_folder, f"{noisy_cost_save}.graphml"))
    cost_noisy.to_csv(join(paths["data_dir"], "raw", "CPH", f"SegmentCosts_noisy_{noisy_cost_save}.csv"))
    
    copyfile(join(paths["input_folder"], save, f"{save}_demand.json"), join(noisy_cost_folder, f"{noisy_cost_save}_demand.json"))
    copyfile(join(paths["input_folder"], save, f"{save}_speeds.json"), join(noisy_cost_folder, f"{noisy_cost_save}_speeds.json"))

In [ ]:
def gen_noisy_demand(d, clean_demand, noisy_demand_save, nodes):
    noisy_demand_folder = join(paths["input_folder"], noisy_demand_save)
    Path(noisy_demand_folder).mkdir(parents=True, exist_ok=True)
    
    demand_noisy, nodes_noise = noisy_demand(clean_demand, d)
    demand_df_noisy = demand_to_od_df(demand_noisy, demand_df, nodes)
    with open(join(noisy_demand_folder, f"{noisy_demand_save}_demand.json"), "w") as fp:
        json.dump({str(k): {int(v_k): v_v for v_k, v_v in v.items()} for k, v in demand_noisy.items()}, fp)
    with open(join(noisy_demand_folder, f"{noisy_demand_save}_node_noise.json"), "w") as fp:
        json.dump(nodes_noise, fp)
    demand_df_noisy.to_csv(join(paths["data_dir"], "raw", "CPH", f"ODData_Complete_noisy_{noisy_demand_save}.csv"), index=False)
    
    copyfile(join(paths["input_folder"], save, f"{save}.graphml"), join(noisy_demand_folder, f"{noisy_demand_save}.graphml"))
    copyfile(join(paths["input_folder"], save, f"{save}_speeds.json"), join(noisy_demand_folder, f"{noisy_demand_save}_speeds.json"))

In [ ]:
def gen_noisy_speeds(d, noisy_speeds_save):
    noisy_speeds_folder = join(paths["input_folder"], noisy_speeds_save)
    Path(noisy_speeds_folder).mkdir(parents=True, exist_ok=True)
    
    speeds_noisy_dict = noisy_speeds({}, d)
    with open(join(noisy_speeds_folder, f"{noisy_speeds_save}_speeds.json"), "w") as fp:
        json.dump(speeds_noisy_dict, fp)
        
    copyfile(join(paths["input_folder"], save, f"{save}.graphml"), join(noisy_speeds_folder, f"{noisy_speeds_save}.graphml"))
    copyfile(join(paths["input_folder"], save, f"{save}_demand.json"), join(noisy_speeds_folder, f"{noisy_speeds_save}_demand.json"))

In [ ]:
delta_noise = 0.20
for i in range(1, 11):
    gen_noisy_speeds(delta_noise, f"{save}_noisy_speeds_{i}")
    gen_noisy_costs(delta_noise, cost, f"{save}_noisy_costs_{i}")
    gen_noisy_demand(delta_noise, demand, f"{save}_noisy_demand_{i}", node_data)